In [ ]:
!pip install -r requirements.txt

In [ ]:
!pip install -e .

In [ ]:
!python scripts/preprocessing/process_biodiscover.py \
    --csv_path ../EPT_final_file_species_level.csv \
    --out_folder data/processed/ept

In [ ]:
!python scripts/01_train_test_split.py \
    --csv_path "data/processed/ept/01_EPT_processed.csv" \
    --target_col "Specimen Weight" \
    --group_col "individual" \
    --n_splits 5 \
    --out_folder "data/processed/ept-biomass"

In [ ]:
import pandas as pd
df = pd.read_csv("data/processed/ept-biomass/01_EPT_processed_5splits_Specimen Weight.csv")
df

In [ ]:
!python scripts/preprocessing/test_dataset_config.py \
    --data_folder "../0_images/" \
    --dataset_config "conf/user_datasets.py" \
    --dataset_name "biodiscover" \
    --csv_path "data/processed/ept-biomass/01_EPT_processed_5splits_Specimen Weight.csv" \
    --label "Specimen Weight" \
    --fold 0

In [1]:
import wandb
wandb.login() #581221dff0063a897a4eff9d285fa871a2c8e7f2

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (2.2.1) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
wandb: Currently logged in as: philipp-rehsen (uni-due). Use `wandb login --relogin` to force relogin


True

In [ ]:
# Only use for testing. Use all folds for actual results
!python scripts/02_train.py \
    --data_folder "../0_images/" \
    --dataset_config "conf/user_datasets.py" \
    --dataset_name "biodiscover" \
    --csv_path "data/processed/ept-biomass/01_EPT_processed_5splits_Specimen Weight.csv" \
    --label "Specimen Weight" \
    --fold 0 \
    --class_map "none" \
    --imsize 224 \
    --batch_size 256 \
    --aug 'flips-rotate-keep-aspect' \
    --load_to_memory 'False' \
    --model 'efficientnet_b0' \
    --opt 'adamw' \
    --max_epochs 10 \
    --min_epochs 0 \
    --early_stopping 'False' \
    --early_stopping_patience 50 \
    --criterion 'l1' \
    --lr 0.004 \
    --auto_lr 'True' \
    --log_dir 'tests' \
    --out_folder 'outputs' \
    --out_prefix '00_ept-biomass_test_run' \
    --deterministic 'True' \
    --precision '16-mixed'

In [2]:
for fold in range(5):
    !python scripts/02_train.py \
        --data_folder "../0_images/" \
        --dataset_config "conf/user_datasets.py" \
        --dataset_name "biodiscover" \
        --csv_path "data/processed/ept-biomass/01_EPT_processed_5splits_Specimen Weight.csv" \
        --label "Specimen Weight" \
        --fold {fold} \
        --class_map 'none' \
        --imsize 224 \
        --batch_size 256 \
        --aug 'flips-rotate-keep-aspect' \
        --load_to_memory 'False' \
        --model 'efficientnet_b0' \
        --opt 'adamw' \
        --max_epochs 50 \
        --min_epochs 0 \
        --early_stopping 'False' \
        --early_stopping_patience 50 \
        --criterion 'l1' \
        --lr 4.786300923226383e-07 \
        --auto_lr 'False' \
        --log_dir 'ept-biomass' \
        --out_folder 'outputs' \
        --out_prefix '01_ept-biomass_run' \
        --deterministic 'True' \
        --precision '16-mixed'

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:619: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (2.2.1) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
INFO:albumentations.check_version:A new version of Albumentations is available: 1.4.8 (you have 1.4.7). Upgrade using: pip install --upgrade albumentations
INFO:timm.models._builder:Loading pretrained weights from Hugging Face hub (timm/efficientnet_b0.ra_in1k)
INFO:timm.models._hub:[timm/efficientnet_b0.ra_in1k] Safe alternative available for 'pytorch_model.bin' (as 'model.safetensors'). Loading weights using safetensors.
None
wandb: Currently logged in as: philipp-rehsen (uni-due). Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.17.0
wandb: Run data is saved locally in ./wandb/run-20

^C
/usr/local/lib/python3.8/dist-packages/lightning/pytorch/trainer/call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...
/usr/local/lib/python3.8/dist-packages/lightning/pytorch/trainer/connectors/checkpoint_connector.py:156: `.test(ckpt_path="best")` is called with Trainer configured with multiple `ModelCheckpoint` callbacks. It will use the best checkpoint path from first checkpoint callback.
Traceback (most recent call last):
  File "scripts/02_train.py", line 29, in <module>
    trainer = main(
  File "scripts/02_train.py", line 23, in main
    return src.TaxonomistModel(
  File "/tf/notebooks/EPT/1_taxonomist/src/taxonomist/taxonomist_model.py", line 461, in train_model
    self._perform_training(trainer, model, dm, resume_ckpt)
  File "/tf/notebooks/EPT/1_taxonomist/src/taxonomist/taxonomist_model.py", line 346, in _perform_training
    trainer.test(model, datamodule=dm, ckpt_path="best")
  File "/usr/local/lib/python3.8/dist-packages/lightning/pytorch/trainer/

In [ ]:
# This just sets the checkpoint as the first (best) model in the directory above, as the unique identifier is always different.
!python scripts/03_predict.py \
    --data_folder "../0_images/" \
    --dataset_config "conf/user_datasets.py" \
    --dataset_name "biodiscover" \
    --csv_path "data/processed/ept/01_EPT_processed_5splits_Final_species_label.csv" \
    --label "Specimen Weight" \
    --fold 0 \
    --class_map "none" \
    --imsize 224 \
    --batch_size 1024 \
    --aug 'none' \
    --out_folder 'outputs' \
    --tta 'False' \
    --out_prefix '' \
    --ckpt_path 'outputs/biodiscover/00_ept-biomass_test_run_efficientnet_b0/f0/00_ept-biomass_test_run_efficientnet_b0_f0_240529-1243-36c0_epoch09_val-loss1.85.ckpt'

In [ ]:
!python scripts/04_group_predictions.py \
    --predictions 'outputs/biodiscover/00_ept-biomass_test_run_efficientnet_b0/f0/predictions/biodiscover_none/_00_ept-biomass_test_run_efficientnet_b0_f0_240529-1243-36c0_epoch09_val-loss1.85_none.csv' \
    --reference_csv "data/processed/ept/01_EPT_processed_5splits_Final_species_label.csv" \
    --reference_target "Specimen Weight" \
    --fold 0 \
    --reference_group "individual" \
    --agg_func "median"

In [ ]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import warnings
def confusion_matrixX(y_true, 
                      y_pred, 
                      classes, 
                      figsize=(15,15), 
                      fonts=(18,10,18), 
                      rotate=False,
                      grid=False,
                      normalize='true'):

    # Check the inputs
    s_true = set(np.unique(y_true))
    s_pred = set(np.unique(y_pred))

    if classes == 'union':
        classes = sorted(list(s_true | s_pred))

    if len((s_true | s_pred) - set(classes)) > 0:
        warnings.warn(f"The inputs contain classes not present in the class list: {(s_true | s_pred) - set(classes)}")

    cm = confusion_matrix(y_true, 
                          y_pred, 
                          labels=classes,
                          normalize=normalize)
    
    if normalize:
        cm = cm*100
    fig, ax = plt.subplots(figsize=figsize)

    mask = np.zeros_like(cm)
    mask[cm==0] = 1
    sns.heatmap(cm, 
                annot=True, 
                annot_kws={"size": fonts[0]},
                fmt='.0f', 
                cmap="YlGnBu",
                xticklabels=classes, 
                yticklabels=classes,
                mask=mask,
                square=True, 
                cbar=False,
                ax=ax)
    
    if grid:
        ax.set_axisbelow(True)
        ax.yaxis.grid(color='gray', linestyle='dashed')
        ax.xaxis.grid(color='gray', linestyle='dashed')

    ax.set_ylabel('True label', size=fonts[2])
    ax.set_xlabel('Predicted label', size=fonts[2])
    ax.yaxis.set_ticks_position('none') 
    ax.xaxis.set_ticks_position('none') 
    
    if rotate:
        if isinstance(rotate, list):
            rot_x = rotate[0]
            rot_y = rotate[1]
        else:
            rot_x = rot_y = rotate
        plt.setp(ax.get_yticklabels(), rotation=rot_x, ha="right", rotation_mode="anchor", size=fonts[1])
        plt.setp(ax.get_xticklabels(), rotation=rot_y, ha="right", rotation_mode="anchor", size=fonts[1])
    else:
        plt.setp(ax.get_yticklabels(), size=fonts[1])
        plt.setp(ax.get_xticklabels(), size=fonts[1])
    plt.show()
        
from sklearn.metrics import classification_report
def classification_reportX(*args, 
                           figsize=None, 
                           fonts=(12,10,10),
                           rotate=True, 
                           bbox_anchor=(1.15,0.8), 
                           **kwargs):
    """Extends classification report by adding an useful plot for the performance across classes"""
    r = classification_report(*args, **kwargs, output_dict=True)
    rdf0 = pd.DataFrame(r)
    rdf = rdf0.T.iloc[:-3,:].sort_values('support',ascending=False)
    fig, ax1 = plt.subplots(figsize=figsize)
    ax2 = ax1.twinx()

    xbar = range(len(rdf))
    ax1.bar(xbar,rdf['support'], alpha=0.2)
    ax2.plot(rdf['f1-score'], 'ro', label='f1-score')
    ax2.plot(rdf['precision'], 'g*', label='precision', alpha=0.5)
    ax2.plot(rdf['recall'], 'b*', label='recall', alpha=0.5)

    ax2.vlines(xbar, rdf['f1-score'], rdf['precision'], 'g')
    ax2.vlines(xbar, rdf['f1-score'], rdf['recall'], 'b')

    ax2.hlines(rdf0['weighted avg']['f1-score'], 0,len(rdf), color='r', linestyle='--', label='weighted f1-score')
    ax2.hlines(rdf0['macro avg']['f1-score'], 0,len(rdf), color='r', linestyle='-.', label='macro f1-score')
    ax2.hlines(rdf0['accuracy']['recall'], 0,len(rdf), color='b', linestyle='--', label='accuracy')
    
    if rotate:
        plt.setp(ax1.get_xticklabels(), 
                 rotation=45, 
                 ha="right", 
                 rotation_mode="anchor",
                 size=fonts[1])

    fig.legend(bbox_to_anchor=bbox_anchor,
               prop={"size": fonts[2]})
    
    ax2.set_ylabel("Score", size=fonts[0])
    ax1.set_ylabel("Support", size=fonts[0])
    plt.setp(ax1.get_yticklabels(),
             size=fonts[0])
    plt.setp(ax2.get_yticklabels(),
             size=fonts[0])
    
    return classification_report(*args, **kwargs)

In [ ]:
import pandas as pd
confusion_df = pd.read_csv('outputs/biodiscover/03_ept_run_efficientnet_b0/f0/predictions/biodiscover_none/_03_ept_run_efficientnet_b0_f0_240529-0859-5fb9_epoch50_val-loss0.82_last_none.csv')

confusion_matrixX(confusion_df['y_true'], confusion_df['y_pred'], 'union')

In [ ]:
!python scripts/04_group_predictions.py \
    --predictions "outputs/biodiscover/aug_resnet18/f0/predictions/biodiscover_none/_aug_resnet18_f0_240524-1427-0065_epoch50_val-loss1.62_last_none.csv" \
    --reference_csv "data/processed/ept/EPT_processed_5splits_Final_species_label.csv" \
    --reference_target "Final_species_label" \
    --fold 0 \
    --reference_group "individual" \
    --agg_func "mode"